In [1]:
%load_ext autotime
%matplotlib inline

time: 1.44 s (started: 2023-07-06 16:05:16 +03:00)


In [2]:
import pandas as pd
import joblib
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import re

time: 3.44 s (started: 2023-07-06 16:05:17 +03:00)


In [3]:
data =  pd.read_csv('Data/SimpleApproach/Imports/Imports2018-19CumulativeGravityHSlevelComplexity.csv')
data.dropna(inplace=True)
data

,refYear,partnerCode,reporterCode,distw_harmonic,contig,gdp_d,gdp_o,gdpcap_d,gdpcap_o,pop_d,...,wto_d,wto_o,refMonth,cmdCode,primaryValue,previous_year_primaryvalue,2HScmdCode,4HScmdCode,pci,density
0,2018,4.0,24.0,7517.0,0.0,1.805323e+07,7.779294e+07,0.486,2.525,37171.921,...,1.0,1.0,9,845420,101822.101,101822.101,84,8454,2.020725,0.006744
1,2018,76.0,24.0,6398.0,0.0,1.916934e+09,7.779294e+07,9.151,2.525,209469.315,...,1.0,1.0,10,845420,203.596,409.456,84,8454,2.020725,0.006744
2,2018,156.0,24.0,11621.0,0.0,1.389482e+10,7.779294e+07,9.905,2.525,1402760.010,...,1.0,1.0,9,845420,995476.726,995476.726,84,8454,2.020725,0.006744
3,2018,410.0,24.0,12763.0,0.0,1.724846e+09,7.779294e+07,33.437,2.525,51585.056,...,1.0,1.0,3,845420,43823.359,43823.359,84,8454,2.020725,0.006744
4,2018,710.0,24.0,2517.0,0.0,4.048421e+08,7.779294e+07,7.005,2.525,57792.519,...,1.0,1.0,8,845420,9222.582,9222.582,84,8454,2.020725,0.006744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43120411,2019,458.0,352.0,11271.0,0.0,3.652764e+08,2.485774e+07,11.433,68.941,31949.789,...,1.0,1.0,8,846721,3629.278,74369.197,84,8467,2.672168,0.034830
43120412,2019,458.0,352.0,11271.0,0.0,3.652764e+08,2.485774e+07,11.433,68.941,31949.789,...,1.0,1.0,10,846721,11405.271,80117.198,84,8467,2.672168,0.034830
43120413,2019,484.0,352.0,7308.0,0.0,1.269404e+09,2.485774e+07,9.950,68.941,127575.531,...,1.0,1.0,1,846721,966.607,9502.791,84,8467,2.672168,0.034830
43120414,2019,484.0,352.0,7308.0,0.0,1.269404e+09,2.485774e+07,9.950,68.941,127575.531,...,1.0,1.0,2,846721,1010.410,8154.953,84,8467,2.672168,0.034830


time: 6min 3s (started: 2023-07-06 16:05:21 +03:00)


In [4]:
def MPE(Y_actual,Y_Predicted):
    mape = np.mean((Y_actual - Y_Predicted)/Y_actual)*100
    return mape

def MAPE(Y_actual,Y_Predicted):
    mpe = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mpe

data['primaryValue'] = data['primaryValue']/1000000
data['gdp_o'] = data['gdp_o']/1000000
data['gdp_d'] = data['gdp_d']/1000000
data['gdpcap_o'] = data['gdpcap_o']/1000000
data['gdpcap_d'] = data['gdpcap_d']/1000000
data['distw_harmonic'] = data['distw_harmonic']/1000

time: 1.29 s (started: 2023-07-06 16:11:24 +03:00)


## RF


In [5]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y


# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, criterion='poisson', max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed:  5.7min remaining: 12.1min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed:  6.5min remaining:  1.2min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  6.6min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    3.2s remaining:   37.2s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    3.8s remaining:    2.6s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    5.1s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   13.8s remaining:  2.7min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   16.0s remaining:   10.7s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   18.9s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.92
R2 on validation data: 0.9
Mean Squared Error: 9.94
Mean Absolute Error: 0.28
Adjusted R^2 Score: 0.9
MAPE:  174183.4799783514
MPE:  -174165.8009836248
time: 7min 26s (started: 2023-07-06 16:11:26 +03:00)


In [7]:
# +density

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, criterion='poisson', max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 14.2min remaining: 30.3min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 15.4min remaining:  2.9min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 16.1min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    2.8s remaining:   31.9s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    3.2s remaining:    2.1s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    3.6s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   12.2s remaining:  2.3min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   13.0s remaining:    8.7s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   13.6s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.92
R2 on validation data: 0.91
Mean Squared Error: 9.74
Mean Absolute Error: 0.28
Adjusted R^2 Score: 0.91
MAPE:  170252.89886300892
MPE:  -170235.47210136204
time: 19min 36s (started: 2023-07-06 16:28:11 +03:00)


In [8]:
# +density

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25
building tree 3 of 25

building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed:  8.3min remaining: 17.7min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed:  8.5min remaining:  1.6min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  8.6min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    3.4s remaining:   38.5s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    4.3s remaining:    2.9s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    5.1s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   17.3s remaining:  3.3min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   19.9s remaining:   13.2s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   21.6s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.92
R2 on validation data: 0.92
Mean Squared Error: 8.33
Mean Absolute Error: 0.29
Adjusted R^2 Score: 0.92
MAPE:  182298.3443246893
MPE:  -182281.16550544734
time: 9min 30s (started: 2023-07-06 16:47:47 +03:00)


In [9]:
# +density+gdpcap

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, criterion='poisson', max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25
building tree 3 of 25

building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 11.0min remaining: 23.3min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 11.4min remaining:  2.2min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 12.3min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    8.3s remaining:  1.6min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   10.3s remaining:    6.9s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   12.3s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   33.5s remaining:  6.4min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   39.3s remaining:   26.2s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   42.1s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.92
R2 on validation data: 0.91
Mean Squared Error: 9.51
Mean Absolute Error: 0.27
Adjusted R^2 Score: 0.91
MAPE:  153366.53868912312
MPE:  -153349.10600603948
time: 14min 3s (started: 2023-07-06 16:57:17 +03:00)


In [10]:
# +density+gdpcap

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d']]
y = data['primaryValue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 14.9min remaining: 31.8min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 15.9min remaining:  3.0min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 16.1min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    5.7s remaining:  1.1min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    7.1s remaining:    4.7s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    9.1s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   13.5s remaining:  2.6min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   22.1s remaining:   14.7s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   26.4s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.93
R2 on validation data: 0.92
Mean Squared Error: 8.21
Mean Absolute Error: 0.29
Adjusted R^2 Score: 0.92
MAPE:  175271.75143270768
MPE:  -175254.64579641938
time: 22min 58s (started: 2023-07-06 17:11:21 +03:00)


In [11]:
# +density+gdpcap+pci

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d','pci']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, criterion='poisson', max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 25.4min remaining: 54.0min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 27.4min remaining:  5.2min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 28.4min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    5.5s remaining:  1.1min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    7.3s remaining:    4.9s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    8.6s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   20.4s remaining:  3.9min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   26.6s remaining:   17.7s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   29.1s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.92
R2 on validation data: 0.91
Mean Squared Error: 9.08
Mean Absolute Error: 0.29
Adjusted R^2 Score: 0.91
MAPE:  153274.29196649854
MPE:  -153257.19209359423
time: 33min 5s (started: 2023-07-06 17:34:20 +03:00)


In [12]:
# +density+gdpcap+pci

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d','pci']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25

building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 11.8min remaining: 25.1min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 12.2min remaining:  2.3min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 12.3min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    1.9s remaining:   22.2s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    2.3s remaining:    1.5s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    2.7s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    8.0s remaining:  1.5min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    8.8s remaining:    5.9s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    9.6s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.92
R2 on validation data: 0.91
Mean Squared Error: 9.54
Mean Absolute Error: 0.32
Adjusted R^2 Score: 0.91
MAPE:  180943.06011787418
MPE:  -180926.83103283026
time: 13min 17s (started: 2023-07-06 18:07:25 +03:00)


In [13]:
data.columns

Index(['refYear', 'partnerCode', 'reporterCode', 'distw_harmonic', 'contig',
       'gdp_d', 'gdp_o', 'gdpcap_d', 'gdpcap_o', 'pop_d', 'pop_o', 'wto_d',
       'wto_o', 'refMonth', 'cmdCode', 'primaryValue',
       'previous_year_primaryvalue', '2HScmdCode', '4HScmdCode', 'pci',
       'density'],
      dtype='object')

time: 6.38 ms (started: 2023-07-06 18:20:42 +03:00)


In [14]:
# +density+gdpcap+pci + No year

X = data[['distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d','pci']]
y = data['primaryValue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, criterion='poisson', max_depth=15, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25
building tree 3 of 25

building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed:  9.6min remaining: 20.3min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed:  9.6min remaining:  1.8min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  9.7min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    1.8s remaining:   20.7s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    2.1s remaining:    1.4s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    2.5s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    7.4s remaining:  1.4min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    8.4s remaining:    5.6s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    9.0s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.89
R2 on validation data: 0.88
Mean Squared Error: 12.19
Mean Absolute Error: 0.33
Adjusted R^2 Score: 0.88
MAPE:  174792.4611476049
MPE:  -174775.79851605132
time: 10min 14s (started: 2023-07-06 18:20:42 +03:00)


In [15]:
# +density+gdpcap+pci + No year

X = data[['distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d','pci']]
y = data['primaryValue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=15, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25

building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed:  9.3min remaining: 19.8min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed:  9.7min remaining:  1.8min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  9.7min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    1.4s remaining:   16.4s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    1.7s remaining:    1.1s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    2.2s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    6.0s remaining:  1.2min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    6.8s remaining:    4.5s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    7.4s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.89
R2 on validation data: 0.88
Mean Squared Error: 12.48
Mean Absolute Error: 0.36
Adjusted R^2 Score: 0.88
MAPE:  217173.11525804058
MPE:  -217157.37030570075
time: 10min 15s (started: 2023-07-06 18:30:57 +03:00)


# Log transformations

In [16]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y

X_train_log = X_train.copy()
y_train_log = np.log(y_train)
X_test_log = X_test.copy()
y_test_log = y_test

X_train_log['gdp_o'] = np.log(X_train_log['gdp_o'])
X_train_log['gdp_d'] = np.log(X_train_log['gdp_d'])
X_train_log['distw_harmonic'] = np.log(X_train_log['distw_harmonic'])

X_test_log['gdp_o'] = np.log(X_test_log['gdp_o'])
X_test_log['gdp_d'] = np.log(X_test_log['gdp_d'])
X_test_log['distw_harmonic'] = np.log(X_test_log['distw_harmonic'])

indices = y_train_log[y_train_log ==-np.inf].index
X_train_log = X_train_log.drop(indices)
y_train_log = y_train_log[y_train_log !=-np.inf]

# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train_log, y_train_log)
y_pred = reg.predict(X_test_log)
y_pred_train = reg.predict(X_train_log)
y_pred = np.exp(y_pred)
y_pred_train = np.exp(y_pred_train)
y_train_log = np.exp(y_train_log)


mse = mean_squared_error(y_test_log,y_pred)
mae = mean_absolute_error(y_test_log,y_pred)
r2 = r2_score(y_test_log, y_pred)
r2_train = r2_score(y_train_log, y_pred_train)

index = y_test_log!=0
mape = MAPE(y_test_log[index], y_pred[index])
mpe = MPE(y_test_log[index], y_pred[index])

print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2.round(2))

print('MSE: ', mse.round(2))
print('MAE: ', mae.round(2))
print('R-square: ', r2.round(2))
print('MAPE: ', mape.round(2))
print('MPE: ', mpe.round(2))

/export/home/chouno/python-installation/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25

building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed:  5.5min remaining: 11.8min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed:  5.6min remaining:  1.1min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  5.7min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    2.7s remaining:   30.9s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    3.0s remaining:    2.0s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    3.6s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   13.2s remaining:  2.5min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   15.4s remaining:   10.3s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   16.8s finished


R2 on training data: 0.24
R2 on validation data: 0.24
MSE:  79.09
MAE:  0.3
R-square:  0.24
MAPE:  6991.59
MPE:  -6918.13
time: 6min 37s (started: 2023-07-06 18:41:12 +03:00)


In [17]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode','density']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y

X_train_log = X_train.copy()
y_train_log = np.log(y_train)
X_test_log = X_test.copy()
y_test_log = y_test

X_train_log['gdp_o'] = np.log(X_train_log['gdp_o'])
X_train_log['gdp_d'] = np.log(X_train_log['gdp_d'])
X_train_log['distw_harmonic'] = np.log(X_train_log['distw_harmonic'])

X_test_log['gdp_o'] = np.log(X_test_log['gdp_o'])
X_test_log['gdp_d'] = np.log(X_test_log['gdp_d'])
X_test_log['distw_harmonic'] = np.log(X_test_log['distw_harmonic'])

indices = y_train_log[y_train_log ==-np.inf].index
X_train_log = X_train_log.drop(indices)
y_train_log = y_train_log[y_train_log !=-np.inf]

# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train_log, y_train_log)
y_pred = reg.predict(X_test_log)
y_pred_train = reg.predict(X_train_log)
y_pred = np.exp(y_pred)
y_pred_train = np.exp(y_pred_train)
y_train_log = np.exp(y_train_log)


mse = mean_squared_error(y_test_log,y_pred)
mae = mean_absolute_error(y_test_log,y_pred)
r2 = r2_score(y_test_log, y_pred)
r2_train = r2_score(y_train_log, y_pred_train)

index = y_test_log!=0
mape = MAPE(y_test_log[index], y_pred[index])
mpe = MPE(y_test_log[index], y_pred[index])

print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2.round(2))

print('MSE: ', mse.round(2))
print('MAE: ', mae.round(2))
print('R-square: ', r2.round(2))
print('MAPE: ', mape.round(2))
print('MPE: ', mpe.round(2))

/export/home/chouno/python-installation/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25

building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed:  6.8min remaining: 14.5min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed:  7.0min remaining:  1.3min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  7.1min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    3.6s remaining:   41.8s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    4.0s remaining:    2.6s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    5.6s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   14.8s remaining:  2.8min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   16.0s remaining:   10.7s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   18.4s finished


R2 on training data: 0.32
R2 on validation data: 0.3
MSE:  72.45
MAE:  0.31
R-square:  0.3
MAPE:  6813.95
MPE:  -6740.56
time: 8min 3s (started: 2023-07-06 18:47:50 +03:00)


In [18]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode','density', 'gdpcap_o','gdpcap_d']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y

X_train_log = X_train.copy()
y_train_log = np.log(y_train)
X_test_log = X_test.copy()
y_test_log = y_test

X_train_log['gdp_o'] = np.log(X_train_log['gdp_o'])
X_train_log['gdp_d'] = np.log(X_train_log['gdp_d'])
X_train_log['distw_harmonic'] = np.log(X_train_log['distw_harmonic'])

X_test_log['gdp_o'] = np.log(X_test_log['gdp_o'])
X_test_log['gdp_d'] = np.log(X_test_log['gdp_d'])
X_test_log['distw_harmonic'] = np.log(X_test_log['distw_harmonic'])

indices = y_train_log[y_train_log ==-np.inf].index
X_train_log = X_train_log.drop(indices)
y_train_log = y_train_log[y_train_log !=-np.inf]

# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train_log, y_train_log)
y_pred = reg.predict(X_test_log)
y_pred_train = reg.predict(X_train_log)
y_pred = np.exp(y_pred)
y_pred_train = np.exp(y_pred_train)
y_train_log = np.exp(y_train_log)


mse = mean_squared_error(y_test_log,y_pred)
mae = mean_absolute_error(y_test_log,y_pred)
r2 = r2_score(y_test_log, y_pred)
r2_train = r2_score(y_train_log, y_pred_train)

index = y_test_log!=0
mape = MAPE(y_test_log[index], y_pred[index])
mpe = MPE(y_test_log[index], y_pred[index])

print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2.round(2))

print('MSE: ', mse.round(2))
print('MAE: ', mae.round(2))
print('R-square: ', r2.round(2))
print('MAPE: ', mape.round(2))
print('MPE: ', mpe.round(2))

/export/home/chouno/python-installation/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25
building tree 3 of 25

building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed:  8.0min remaining: 17.0min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed:  8.1min remaining:  1.6min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  8.2min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    3.0s remaining:   34.1s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    3.8s remaining:    2.5s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    4.3s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   10.6s remaining:  2.0min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   11.9s remaining:    7.9s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   13.7s finished


R2 on training data: 0.3
R2 on validation data: 0.3
MSE:  72.74
MAE:  0.31
R-square:  0.3
MAPE:  6406.3
MPE:  -6332.97
time: 9min 14s (started: 2023-07-06 18:55:53 +03:00)


In [19]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode','density', 'gdpcap_o','gdpcap_d','pci']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y

X_train_log = X_train.copy()
y_train_log = np.log(y_train)
X_test_log = X_test.copy()
y_test_log = y_test

X_train_log['gdp_o'] = np.log(X_train_log['gdp_o'])
X_train_log['gdp_d'] = np.log(X_train_log['gdp_d'])
X_train_log['distw_harmonic'] = np.log(X_train_log['distw_harmonic'])

X_test_log['gdp_o'] = np.log(X_test_log['gdp_o'])
X_test_log['gdp_d'] = np.log(X_test_log['gdp_d'])
X_test_log['distw_harmonic'] = np.log(X_test_log['distw_harmonic'])

indices = y_train_log[y_train_log ==-np.inf].index
X_train_log = X_train_log.drop(indices)
y_train_log = y_train_log[y_train_log !=-np.inf]

# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train_log, y_train_log)
y_pred = reg.predict(X_test_log)
y_pred_train = reg.predict(X_train_log)
y_pred = np.exp(y_pred)
y_pred_train = np.exp(y_pred_train)
y_train_log = np.exp(y_train_log)


mse = mean_squared_error(y_test_log,y_pred)
mae = mean_absolute_error(y_test_log,y_pred)
r2 = r2_score(y_test_log, y_pred)
r2_train = r2_score(y_train_log, y_pred_train)

index = y_test_log!=0
mape = MAPE(y_test_log[index], y_pred[index])
mpe = MPE(y_test_log[index], y_pred[index])

print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2.round(2))

print('MSE: ', mse.round(2))
print('MAE: ', mae.round(2))
print('R-square: ', r2.round(2))
print('MAPE: ', mape.round(2))
print('MPE: ', mpe.round(2))

/export/home/chouno/python-installation/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25
building tree 3 of 25

building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed:  9.2min remaining: 19.6min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed:  9.3min remaining:  1.8min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  9.5min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    3.8s remaining:   43.8s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    4.1s remaining:    2.7s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    5.1s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   16.1s remaining:  3.1min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   16.9s remaining:   11.2s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   19.5s finished


R2 on training data: 0.23
R2 on validation data: 0.24
MSE:  78.57
MAE:  0.31
R-square:  0.24
MAPE:  6261.28
MPE:  -6187.97
time: 10min 37s (started: 2023-07-06 19:05:08 +03:00)


In [20]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode','density','pci']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y

X_train_log = X_train.copy()
y_train_log = np.log(y_train)
X_test_log = X_test.copy()
y_test_log = y_test

X_train_log['gdp_o'] = np.log(X_train_log['gdp_o'])
X_train_log['gdp_d'] = np.log(X_train_log['gdp_d'])
X_train_log['distw_harmonic'] = np.log(X_train_log['distw_harmonic'])

X_test_log['gdp_o'] = np.log(X_test_log['gdp_o'])
X_test_log['gdp_d'] = np.log(X_test_log['gdp_d'])
X_test_log['distw_harmonic'] = np.log(X_test_log['distw_harmonic'])

indices = y_train_log[y_train_log ==-np.inf].index
X_train_log = X_train_log.drop(indices)
y_train_log = y_train_log[y_train_log !=-np.inf]

# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train_log, y_train_log)
y_pred = reg.predict(X_test_log)
y_pred_train = reg.predict(X_train_log)
y_pred = np.exp(y_pred)
y_pred_train = np.exp(y_pred_train)
y_train_log = np.exp(y_train_log)


mse = mean_squared_error(y_test_log,y_pred)
mae = mean_absolute_error(y_test_log,y_pred)
r2 = r2_score(y_test_log, y_pred)
r2_train = r2_score(y_train_log, y_pred_train)

index = y_test_log!=0
mape = MAPE(y_test_log[index], y_pred[index])
mpe = MPE(y_test_log[index], y_pred[index])

print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2.round(2))

print('MSE: ', mse.round(2))
print('MAE: ', mae.round(2))
print('R-square: ', r2.round(2))
print('MAPE: ', mape.round(2))
print('MPE: ', mpe.round(2))

/export/home/chouno/python-installation/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25

building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed:  7.6min remaining: 16.1min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed:  7.8min remaining:  1.5min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  8.1min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    3.7s remaining:   42.7s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    4.0s remaining:    2.7s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    5.7s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   15.7s remaining:  3.0min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   16.4s remaining:   10.9s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   22.5s finished


R2 on training data: 0.29
R2 on validation data: 0.33
MSE:  69.73
MAE:  0.3
R-square:  0.33
MAPE:  6736.19
MPE:  -6662.77
time: 9min 16s (started: 2023-07-06 19:15:45 +03:00)
